# Predict binding affinity from a protein-ligand crystal structure

In [3]:
# Packages installed in my conda environment: Deepchem, rdkit, pytorch, pytorch-geometric, tensorflow, pdbfixer, numpy <1.25
# Deep Learning for the Life Sciences by Bharath Ramsundar, Peter Eastman, Patrick Walters and Vijay Pande
# pdbbind dataset: Cheng, T.J. et al. J. Chem. Inf. Model., 2009, 49, 1079-1093. (PDBbind v.2009)

### How to install Libraries in yml file

 - 일반적으로 yml/yaml file에 있는 라이브러리들은 `requirements.txt` 파일 형식으로 작성되어 있다.
 - `pip` 는 이 파일을 읽어 필요한 라이브러리를 한번에 설치할 수 있다.


 - `pip install -r requirements.txt` 이 명령은 `requirements.txt` 파일에 있는 각 라이브러리와 해당 버전을 자동으로 가져와 설치한다.
 * `requirements.txt` 파일이 있는 디렉토리로 이동한 후 명령을 실행해야 한다.

In [4]:
pip install -r env_commom.yml

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'env_commom.yml'
Note: you may need to restart the kernel to use updated packages.


In [1]:
import deepchem as dc

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/dbsejrgus226/miniconda3/envs/deepchem/lib/python3.7/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
featurizer = dc.feat.RdkitGridFeaturizer(voxel_width=2.0,
                                         feature_types=["ecfp", "splif", "salt_bridge", "hbond"],
                                         flatten = True,
                                         sanitize = True)

In [3]:
tasks, datasets, transformers = dc.molnet.load_pdbbind(featurizer=featurizer, reload=False, set_name = "core")

PermissionError: [Errno 13] Permission denied: '/tmp/pdbbind_v2013_core_set.tar.gz'

In [ ]:
tasks

['-logKd/Ki']

In [ ]:
transformers

In [ ]:
train, vaild, test = datasets

In [ ]:
print(train.X.dtype)
print(train.y.dtype)
print(train.w.dtype)

int8
float64
float32


In [3]:
x_train = train.X.astype("float32")
y_train = train.y.astype("float32")
w_train = train.w.astype("float32")

x_test = test.X.astype("float32")
y_test = test.y.astype("float32")
w_test = test.w.astype("float32")

NameError: name 'train' is not defined

In [28]:
x_train.dtype

dtype('float32')

In [29]:
train.X.shape

(154, 18432)

In [39]:
train_dataset = dc.data.NumpyDataset(x_train, y_train, w_train)
test_dataset = dc.data.NumpyDataset(x_test, y_test, w_test)

In [40]:
train_dataset.X.shape

(154, 18432)

In [43]:
train_dataset.y.shape

(154,)

In [63]:
from deepchem.models import DTNNModel

In [65]:
model = DTNNModel(n_tasks=1,
                  n_embedding=20,
                  n_distance=100,
                  learning_rate=1.0,
                  mode="regression",
                  )

In [68]:
model.fit(train_dataset, nb_epoch=250)

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

In [ ]:
train_score = model.evaluate(train_dataset, [metric], transformers)
print(train_score)

In [ ]:
test_score = model.evaluate(test_dataset, [metric], transformers)
print(test_score)

In [ ]:
test_dataset.X.shape

In [ ]:
prediction = model.predict_on_batch(test_dataset.X)
prediction[:5]

In [ ]:
test_dataset.y[:5]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
prediction.shape

In [ ]:
prediction = prediction.reshape(20)
prediction.shape

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax = sns.scatterplot(x=test_dataset.y, y=prediction)
ax.set_title('pdbbind dataset')
ax.set_xlabel('True')
ax.set_ylabel('prediction')